# Chapter 10. Introduction to Artificial Neural Networks with Keras
## Building Complex Models Using the Functional API

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [25]:
X_train_A, X_train_B = X_train_scaled[:, :5], X_train_scaled[:, 2:]
X_valid_A, X_valid_B = X_valid_scaled[:, :5], X_valid_scaled[:, 2:]
X_test_A, X_test_B = X_test_scaled[:, :5], X_test_scaled[:, 2:]

In [40]:
print(X_train_A.shape, X_train_B.shape)

(11610, 5) (11610, 6)


In [3]:
from tensorflow import keras

In [50]:
# Some features to the wide path and some other to deep path
input_A = keras.layers.Input(shape=[5])
input_B = keras.layers.Input(shape=[6])
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [51]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [52]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 4.2145 - val_loss: 1.1197
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.9482 - val_loss: 0.7521
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7013 - val_loss: 0.6638
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6146 - val_loss: 0.6177
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5963 - val_loss: 0.5828
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5552 - val_loss: 0.5571
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5546 - val_loss: 0.5377
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5113 - val_loss: 0.5224
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4870 - val_loss: 0.5105
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4986 - val_loss: 0.5009

In [46]:
mse = model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 0s 997us/step - loss: 0.4677


## Building Dynamic Models Using the Subclassing API

In [47]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
model = WideAndDeepModel()

## Saving and Restoring a Model

In [53]:
model.save('my_keras_model.h5')

In [54]:
model = keras.models.load_model('my_keras_model.h5')

## Using Callbacks

In [55]:
# early stopping
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid),
                   callbacks=[checkpoint_cb])

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4490 - val_loss: 0.4571
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4466 - val_loss: 0.4555
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4443 - val_loss: 0.4531
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4422 - val_loss: 0.4516
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4401 - val_loss: 0.4493
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4382 - val_loss: 0.4477
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4363 - val_loss: 0.4472
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4347 - val_loss: 0.4450
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4329 - val_loss: 0.4442
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4314 - val_loss: 0.4426

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [62]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

In [66]:
run_logdir = get_run_logdir()

In [67]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid),
                   callbacks=[tensorboard_cb])

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3658 - val_loss: 0.3794
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3656 - val_loss: 0.3806
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3648 - val_loss: 0.3797
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3644 - val_loss: 0.3790
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3634 - val_loss: 0.3780
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3629 - val_loss: 0.3767
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3623 - val_loss: 0.3763
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3618 - val_loss: 0.3763
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3612 - val_loss: 0.3756
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3605 - val_loss: 0.3758